In [1]:

with open('doc3.txt', encoding="mbcs") as f:
    lines =f.readlines()

In [2]:
data=[]
k =0
for i in range(len(lines)):
    

    if(lines[i][0:12]=='Section name'):

        entry={}
        entry["Sec"]=lines[i][14:-1]
        entry["Para"] = lines[i+1][10:-1]
        j=0
        while(lines[i+2+j][0:10] !='Key words:'):
            j+=1

        entry["Keys"] = lines[i+2+j][14:-4].split(',')

        data.append(entry)
        k+=1
        
        

In [3]:
data

[{'Sec': ' Introduction',
  'Para': 'In deep reinforcement learning (DRL), ample methods exist to successfully handle large state spaces, but methods to handle large discrete action spaces (LDAS) remain scarce [Dulac-Arnold et al., 2021]. Still, LDAS often arise when applying DRL to real-world applications, e.g., for recommender systems [Afsar et al., 2022], portfolio optimization [Pigorsch and SchÃ¤fer, 2021], or inventory replenishment problems [Boute et al., 2022]. The decision space for such problems is often discrete and suffers from a curse of dimensionality, e.g., managing the inventory replenishment for a group of N products with each having G different order levels yields an action space of size GN . Off-the-shelf DRL algorithms â€“ e.g., Deep Q-Networks (DQN) [Mnih et al., 2013], Deep Policy Gradients (DPG) [Silver et al., 2014], or Proximal Policy Optimization (PPO) [Schulman et al., 2017] â€“ fail to handle such LDAS, as they require in- or output nodes for each discrete ac

In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, utils
utils.logging.set_verbosity_error()  # Suppress standard warnings
model = AutoModelForMaskedLM.from_pretrained("allenai/longformer-base-4096", output_attentions=True)

In [5]:
from transformers import LongformerTokenizer

tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
tokenizer("Hello world")["input_ids"]

tokenizer(" Hello world")["input_ids"]

[0, 20920, 232, 2]

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

In [155]:
txt = data[0]["Sec"]+data[0]["Para"][0:350]
import torch
l = len(tokenizer.encode(txt,return_tensors='pt')[0])
tokenizer.encode(txt,return_tensors='pt')[0]
attentions = model(tokenizer.encode(txt,return_tensors='pt'))[-1]
att=[]
for i in range(12):
    a=torch.zeros([1,12,l,l])
    for j in range(12):
        for k in range(l):
            a[0][j][k] = attentions[i][0][j][k][256-k:256+l-k]
    att.append(a)


In [156]:
inputs = tokenizer.encode(txt, return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

In [157]:
key_tokens =[]
key_arr = data[0]["Keys"]
key_arr.insert(0,data[0]["Sec"])
for key in data[0]["Keys"]:
    toks = tokenizer.encode(key, return_tensors='pt')[0]
    for i in range(len(toks)-2):
        key_tokens.append(toks[i+1])
key_tokens =torch.Tensor(key_tokens)

In [161]:
indices=[]
token_val = tokenizer.encode(txt,return_tensors='pt')[0]
for keyt in key_tokens:
    try:
        locs=(token_val==keyt).nonzero(as_tuple=True)[0]
        for loc in locs:
            indices.append(loc)
    except:
        print((token_val==keyt).nonzero(as_tuple=True)[0][0])
indices =torch.sort(torch.Tensor(indices))[0]

In [8]:
import torch
atts=[]
tokss=[]
sump=[]
for i1 in range(len(data)):
         
        
    txt = str(data[i1]["Sec"]+data[i1]["Para"][0:200])
    key_tokens=[]
    key_arr = list(data[i1]["Keys"])
    
    key_arr.insert(0,str(data[i1]["Sec"]))
    for key in key_arr:
        toks = tokenizer.encode(key, return_tensors='pt')[0]
        for i in range(len(toks)-2):
            key_tokens.append(toks[i+1])
    key_tokens =torch.Tensor(key_tokens)
    indices=[]
    token_val = tokenizer.encode(txt,return_tensors='pt')[0]
    for keyt in key_tokens:
        try:
            locs=(token_val==keyt).nonzero(as_tuple=True)[0]
            for loc in locs:
                indices.append(loc)
        except:
            print((token_val==keyt).nonzero(as_tuple=True)[0][0])

    indices =torch.sort(torch.Tensor(indices))[0]

    
    import torch
    l = len(indices)
    # token_val = tokenizer.encode(txt,return_tensors='pt')[0]
    tokenizer.encode(txt,return_tensors='pt')[0]
    attentions = model(tokenizer.encode(txt,return_tensors='pt'))[-1]
    att=[]
    for i in range(12):
        a=torch.zeros([1,12,l,l])
        for j in range(12):
            for k in range(l):
                for m in range(l):  
                    try:a[0][j][k][m] = 10*attentions[i][0][j][int(indices[k])][int(256+indices[m]-indices[k])]
                    except: print(j,k,m)
        att.append(a)
    tokens = tokenizer.convert_ids_to_tokens(token_val[[list(indices.type(torch.int))]])
    sum=0
    for i in range(len(att)):
        for j in range(att[0].shape[1]):
            sum+= att[i][0][j][0:len(tokenizer.encode(data[i1]["Sec"],return_tensors='pt')[0])-2].sum()
    sump.append(sum)
    atts.append(att)
    tokss.append(tokens)

: 

: 

In [488]:
sump

[tensor(117.3403, grad_fn=<AddBackward0>),
 tensor(892.3118, grad_fn=<AddBackward0>),
 tensor(1095.5081, grad_fn=<AddBackward0>),
 tensor(1246.2035, grad_fn=<AddBackward0>),
 tensor(668.2901, grad_fn=<AddBackward0>),
 tensor(2080.9136, grad_fn=<AddBackward0>),
 tensor(118.7301, grad_fn=<AddBackward0>),
 tensor(187.2670, grad_fn=<AddBackward0>),
 tensor(5525.7007, grad_fn=<AddBackward0>),
 tensor(7596.7734, grad_fn=<AddBackward0>)]

In [480]:
indices2= []
for keyt in key_tokens:

    locs=(token_val==keyt).nonzero(as_tuple=True)[0]
    for loc in locs:
        indices2.append(loc)

In [481]:
indices2

[tensor(1),
 tensor(2),
 tensor(3),
 tensor(16),
 tensor(47),
 tensor(4),
 tensor(17),
 tensor(48),
 tensor(5),
 tensor(6),
 tensor(7),
 tensor(8),
 tensor(9),
 tensor(2),
 tensor(3),
 tensor(16),
 tensor(47),
 tensor(4),
 tensor(17),
 tensor(48),
 tensor(18),
 tensor(49),
 tensor(19),
 tensor(28),
 tensor(50),
 tensor(30),
 tensor(31),
 tensor(32),
 tensor(44),
 tensor(18),
 tensor(49),
 tensor(13),
 tensor(19),
 tensor(28),
 tensor(50),
 tensor(13),
 tensor(13)]

In [452]:
list(indices2.type(torch.int))

[tensor(1, dtype=torch.int32),
 tensor(2, dtype=torch.int32),
 tensor(3, dtype=torch.int32),
 tensor(3, dtype=torch.int32),
 tensor(4, dtype=torch.int32),
 tensor(4, dtype=torch.int32),
 tensor(5, dtype=torch.int32),
 tensor(6, dtype=torch.int32),
 tensor(7, dtype=torch.int32),
 tensor(8, dtype=torch.int32),
 tensor(9, dtype=torch.int32),
 tensor(13, dtype=torch.int32),
 tensor(13, dtype=torch.int32),
 tensor(13, dtype=torch.int32),
 tensor(15, dtype=torch.int32),
 tensor(16, dtype=torch.int32),
 tensor(16, dtype=torch.int32),
 tensor(17, dtype=torch.int32),
 tensor(17, dtype=torch.int32),
 tensor(18, dtype=torch.int32),
 tensor(18, dtype=torch.int32),
 tensor(19, dtype=torch.int32),
 tensor(19, dtype=torch.int32),
 tensor(28, dtype=torch.int32),
 tensor(28, dtype=torch.int32),
 tensor(30, dtype=torch.int32),
 tensor(31, dtype=torch.int32),
 tensor(32, dtype=torch.int32),
 tensor(44, dtype=torch.int32),
 tensor(46, dtype=torch.int32),
 tensor(47, dtype=torch.int32),
 tensor(47, dtype=t

In [439]:
indices1=[]
token_val = tokenizer.encode(txt,return_tensors='pt')[0]
for keyt in key_tokens:
    try:
        locs=(token_val==keyt).nonzero(as_tuple=True)[0]
        for loc in locs:
            indices1.append(loc)
    except:
        print((token_val==keyt).nonzero(as_tuple=True)[0][0])

In [451]:
indices2

tensor([ 1.,  2.,  3.,  3.,  4.,  4.,  5.,  6.,  7.,  8.,  9., 13., 13., 13.,
        15., 16., 16., 17., 17., 18., 18., 19., 19., 28., 28., 30., 31., 32.,
        44., 46., 47., 47., 48., 48., 49., 49., 50., 50.])

In [449]:
indices2 = torch.sort(torch.Tensor(indices1))[0]

In [436]:
tokenizer.encode(txt,return_tensors='pt')[0]

tensor([    0, 19020,  6103,   179,  3693, 30536, 13123,    13, 31684,  1258,
         7908,     6,    98,    12,  4155, 40846,   179,  3693,  2777,  3092,
           11,     5,  6492,   443,  4412,  9115,     7,   167,  3092,     9,
          163, 18854,   651,   646, 30504,  2614,  4400,  1076,   482,   954,
         8174, 11050,     7,     5,  6424,     9, 40846,   179,  3693,  2777,
         3092,   939,     2])

In [432]:
indices

tensor([ 1.,  2.,  3.,  3.,  4.,  4.,  5.,  6.,  7.,  8.,  9., 13., 13., 13.,
        15., 16., 16., 17., 17., 18., 18., 19., 19., 28., 28., 30., 31., 32.,
        44., 46., 47., 47., 48., 48., 49., 49., 50., 50.])

In [354]:
data[i1]["Keys"]

[' discriminative language models',
 ' BERT series',
 ' expertise',
 ' natural language',
 ' embedding backbones',
 ' downstream tasks',
 ' pre-trained models',
 ' domain-specific data',
 ' fine-tuning',
 ' prompt tuning']

In [361]:
i1=9

key_arr = data[i1]["Keys"]
key_arr.insert(0,data[i1]["Sec"])
for key in data[i1]["Keys"]:
    toks = tokenizer.encode(key, return_tensors='pt')[0]
    for i in range(len(toks)-2):
        key_tokens.append(toks[i+1])
key_tokens =torch.Tensor(key_tokens)
indices=[]
token_val = tokenizer.encode(txt,return_tensors='pt')[0]
for keyt in key_tokens:
    try:
        locs=(token_val==keyt).nonzero(as_tuple=True)[0]
        for loc in locs:
            indices.append(loc)
    except:
        print((token_val==keyt).nonzero(as_tuple=True)[0][0])
indices =torch.sort(torch.Tensor(indices))[0]
txt = data[i1]["Sec"]+data[i1]["Para"][0:200]
import torch
l = len(indices)
token_val = tokenizer.encode(txt,return_tensors='pt')[0]
tokenizer.encode(txt,return_tensors='pt')[0]
attentions = model(tokenizer.encode(txt,return_tensors='pt'))[-1]
att=[]
for i in range(12):
    a=torch.zeros([1,12,l,l])
    for j in range(12):
        for k in range(l):
            for m in range(l):  
                try:a[0][j][k][m] = 10*attentions[i][0][j][int(indices[k])][int(256+indices[m]-indices[k])]
                except: print(j,k,m)
    att.append(a)
tokens = tokenizer.convert_ids_to_tokens(token_val[[list(indices.type(torch.int))]])
sum=0
for i in range(len(att)):
    for j in range(att[0].shape[1]):
        sum+= att[i][0][j][0:len(tokenizer.encode(data[i1]["Sec"],return_tensors='pt')[0])-2].sum()
sump.append(sum)

AttributeError: 'Tensor' object has no attribute 'append'

In [304]:
tokens

['Ġ']

In [429]:
data=[]
k =0
for i in range(len(lines)):
    

    if(lines[i][0:12]=='Section name'):
        if(k==7):
            print(lines[i][0:12])
            print(lines[i+1][10:-1])
            print(lines[i+2])
        entry={}
        entry["Sec"]=lines[i][14:-1]
        entry["Para"] = lines[i+1][10:-1]
        j=0
        while(lines[i+2+j][0:10] !='Key words:'):
            j+=1
        if(k==7):
            print(lines[i+2+j][10:-1].split(','))
        entry["Keys"] = lines[i+2+j][10:-1].split(',')

        data.append(entry)
        k+=1
        

Section name
 Recommendation systems play a critical role in assisting users in finding relevant and personalized items or content. With the emergence of Large Language Models (LLMs) in Natural Language Processing (NLP), there has been a growing interest in harnessing the power of these models to enhance recommendation systems.The key advantage of incorporating LLMs into recommendation systems lies in their ability to extract high-quality representations of textual features and leverage the extensive external knowledge encoded within them [Liu et al., 2023b].
Key words: Large Language Models, Natural Language Processing, enhance recommendation systems, textual features, harnessing the power

[' Large Language Models', ' Natural Language Processing', ' enhance recommendation systems', ' textual features', ' harnessing the power']


In [277]:
key_arr

[' Introduction',
 ' Large Language Models',
 ' Natural Language Processing',
 ' enhance recommendation systems',
 ' textual features',
 ' harnessing the power']

In [430]:
import numpy as np
i1 =9
key_tokens =[]
key_arr = list(data[i1]["Keys"])
key_arr.insert(0,data[i1]["Sec"])
for key in key_arr:
    toks = tokenizer.encode(key, return_tensors='pt')[0]
    for i in range(len(toks)-2):
        key_tokens.append(toks[i+1])
key_tokens =torch.Tensor(key_tokens)
indices=[]
token_val = tokenizer.encode(txt,return_tensors='pt')[0]
for keyt in key_tokens:
    try:
        locs=(token_val==keyt).nonzero(as_tuple=True)[0]
        for loc in locs:
            indices.append(loc)
    except:
        continue
        print((token_val==keyt).nonzero(as_tuple=True))
indices =torch.sort(torch.Tensor(indices))[0]
txt = data[i1]["Sec"]+data[i1]["Para"][0:200]
import torch
l = len(indices)
token_val = tokenizer.encode(txt,return_tensors='pt')[0]

attentions = model(tokenizer.encode(txt,return_tensors='pt'))[-1]
att=[]
for i in range(12):
    a=torch.zeros([1,12,l,l])
    for j in range(12):
        for k in range(l):
            for m in range(l):  
                try:a[0][j][k][m] = 10*attentions[i][0][j][int(indices[k])][int(256+indices[m]-indices[k])]
                except: print(j,k,m)
    att.append(a)
tokens = tokenizer.convert_ids_to_tokens(token_val[[list(indices.type(torch.int))]])
sum=0
for i in range(len(att)):
    for j in range(att[0].shape[1]):
        sum+= att[i][0][j][0:len(tokenizer.encode(data[i1]["Sec"],return_tensors='pt')[0])-2].sum()


In [431]:
tokens

['ĠDisc',
 'rim',
 'in',
 'in',
 'ative',
 'ative',
 'ĠLL',
 'Ms',
 'Ġfor',
 'ĠRecommend',
 'ation',
 '-',
 '-',
 '-',
 'Ġdiscrim',
 'in',
 'in',
 'ative',
 'ative',
 'Ġlanguage',
 'Ġlanguage',
 'Ġmodels',
 'Ġmodels',
 'Ġmodels',
 'Ġmodels',
 'ĠB',
 'ERT',
 'Ġseries',
 'Ġexpertise',
 'Ġdiscrim',
 'in',
 'in',
 'ative',
 'ative',
 'Ġlanguage',
 'Ġlanguage',
 'Ġmodels',
 'Ġmodels']

In [419]:
sum

tensor(131.9956, grad_fn=<AddBackward0>)

In [411]:
len(tokens)

38

In [402]:
(token_val==3092).nonzero(as_tuple=True)[0]

tensor([19, 28, 50])

In [408]:
key_tokens.sort()

torch.return_types.sort(
values=tensor([1.2000e+01, 1.2000e+01, 1.2000e+01, 1.3000e+01, 1.2400e+02, 1.5400e+02,
        1.6300e+02, 1.7900e+02, 1.7900e+02, 4.1400e+02, 6.5100e+02, 1.1980e+03,
        1.2580e+03, 1.6320e+03, 2.0510e+03, 2.7770e+03, 2.7770e+03, 3.0920e+03,
        3.0920e+03, 3.6930e+03, 3.6930e+03, 6.1030e+03, 6.4240e+03, 8.5580e+03,
        1.1170e+04, 1.1303e+04, 1.3123e+04, 1.4175e+04, 1.4302e+04, 1.8561e+04,
        1.8854e+04, 1.9020e+04, 2.3830e+04, 2.4641e+04, 2.8927e+04, 3.0536e+04,
        3.1684e+04, 3.3183e+04, 3.4394e+04, 4.0846e+04]),
indices=tensor([27, 31, 35,  6, 22, 37, 14,  2, 10, 33, 16, 26,  8, 18, 34, 19, 12, 13,
        29,  3, 11,  1, 17, 25, 30, 21,  5, 32, 38, 24, 15,  0, 28, 36, 39,  4,
         7, 20, 23,  9]))

In [390]:
tokens

['ĠDisc',
 'rim',
 'in',
 'in',
 'ative',
 'ative',
 'ĠLL',
 'Ms',
 'Ġfor',
 'ĠRecommend',
 'ation',
 '-',
 '-',
 '-',
 'Ġdiscrim',
 'in',
 'in',
 'ative',
 'ative',
 'Ġlanguage',
 'Ġlanguage',
 'Ġmodels',
 'Ġmodels',
 'Ġmodels',
 'Ġmodels',
 'ĠB',
 'ERT',
 'Ġseries',
 'Ġexpertise',
 'Ġdiscrim',
 'in',
 'in',
 'ative',
 'ative',
 'Ġlanguage',
 'Ġlanguage',
 'Ġmodels',
 'Ġmodels']

In [400]:
tokenizer.encode(' models',return_tensors="pt")

tensor([[   0, 3092,    2]])

In [403]:
tokenizer.convert_ids_to_tokens(3092)

'Ġmodels'

In [387]:
key_tokens

tensor([4.0846e+04, 1.7900e+02, 3.6930e+03, 2.7770e+03, 3.0920e+03, 1.6300e+02,
        1.8854e+04, 6.5100e+02, 6.4240e+03, 1.6320e+03, 2.7770e+03, 3.3183e+04,
        1.1303e+04, 1.2400e+02, 3.4394e+04, 1.8561e+04, 8.5580e+03, 1.1980e+03,
        1.2000e+01, 2.3830e+04, 3.0920e+03, 1.1170e+04, 1.2000e+01, 1.4175e+04,
        4.1400e+02, 2.0510e+03, 1.2000e+01, 2.4641e+04, 1.5400e+02, 1.4302e+04,
        2.8927e+04])

In [376]:
key_arr

[' Discriminative LLMs for Recommendation',
 ' discriminative language models',
 ' BERT series',
 ' expertise',
 ' natural language',
 ' embedding backbones',
 ' downstream tasks',
 ' pre-trained models',
 ' domain-specific data',
 ' fine-tuning',
 ' prompt tuning']

In [344]:
data[i1]["Keys"]

[' vanilla actor-critic algorithm',
 ' k-nearest neighbors',
 ' learned action representation',
 ' MinMax',
 ' hyperparameter tuning',
 ' benchmarks',
 ' baseline',
 ' state-of-the-art benchmarks']

In [345]:
tokens

['Ġbenchmarks',
 'Ġbenchmarks',
 'Ġvanilla',
 'Ġactor',
 '-',
 '-',
 '-',
 '-',
 '-',
 'crit',
 'ic',
 'Ġalgorithm',
 'ĠMin',
 'Max',
 'Ġk',
 '-',
 '-',
 '-',
 '-',
 '-',
 'ne']

In [264]:
tokenizer.convert_ids_to_tokens(token_val[[list(indices.type(torch.int))]])

IndexError: index 43 is out of bounds for dimension 0 with size 37

In [266]:
txt

' Introduction Recommendation systems play a critical role in assisting users in finding relevant and personalized items or content. With the emergence of Large Language Models (LLMs) in Natural Language Processing'

In [260]:
indices

tensor([21., 22., 43.])

tensor([], dtype=torch.int64)

In [246]:
tokenizer.encode(txt, return_tensors='pt')[0]

tensor([    0, 19020,  6103,   179,  3693, 30536, 13123,    13, 31684,  1258,
         7908,     6,    98,    12,  4155, 40846,   179,  3693,  2777,  3092,
           11,     5,  6492,   443,  4412,  9115,     7,   167,  3092,     9,
          163, 18854,   651,   646, 30504,  2614,  4400,  1076,   482,   954,
         8174, 11050,     7,     5,  6424,     9, 40846,   179,  3693,  2777,
         3092,   939,     2])

In [243]:
txt

' Discriminative LLMs for Recommendation Indeed, so-called discriminative language models in the recommendation area mainly refer to those models of BERT series [Devlin et al., 2019]. Due to the expertise of discriminative language models i'

In [190]:
data[7]["Sec"]+data[7]["Para"][0:200]

' Introduction Recommendation systems play a critical role in assisting users in finding relevant and personalized items or content. With the emergence of Large Language Models (LLMs) in Natural Language Processing'

In [191]:
data[7]["Keys"]

[' Introduction',
 ' Large Language Models',
 ' Natural Language Processing',
 ' enhance recommendation systems',
 ' textual features',
 ' harnessing the power']

In [185]:
sump[7]

tensor(0.)

In [164]:
tokens = tokenizer.convert_ids_to_tokens(token_val[[list(indices.type(torch.int))]])

In [165]:
tokens

['ĠIntroduction',
 'Ġdeep',
 'Ġreinforcement',
 'Ġlearning',
 'Ġlarge',
 'Ġspaces',
 'Ġlarge',
 'Ġdiscrete',
 'Ġaction',
 'Ġspaces',
 '-',
 '-',
 'Ġrecomm',
 'ender',
 'Ġsystems',
 'Ġportfolio',
 'Ġoptimization']

In [167]:
len(tokenizer.encode(data[0]["Sec"],return_tensors='pt')[0])-2

1

In [486]:
from bertviz import head_view
head_view(atts[7], tokss[7])

<IPython.core.display.Javascript object>

In [257]:
tokss[5]

['Ġ']

In [175]:
tokens

['ĠIntroduction',
 'Ġdeep',
 'Ġreinforcement',
 'Ġlearning',
 'Ġlarge',
 'Ġspaces',
 'Ġlarge',
 'Ġdiscrete',
 'Ġaction',
 'Ġspaces',
 '-',
 '-',
 'Ġrecomm',
 'ender',
 'Ġsystems',
 'Ġportfolio',
 'Ġoptimization']

In [177]:
sum=0
for i in range(len(att)):
    for j in range(att[0].shape[1]):
        sum+= att[i][0][j][0:len(tokenizer.encode(data[0]["Sec"],return_tensors='pt')[0])-2].sum()


In [178]:
sum

tensor(135.1872, grad_fn=<AddBackward0>)

In [176]:
len(tokenizer.encode(data[0]["Sec"],return_tensors='pt')[0])

3

In [174]:
sum

tensor(339.1735, grad_fn=<AddBackward0>)

In [172]:
att[0].shape

torch.Size([1, 12, 17, 17])

In [119]:
tokenizer.encode(txt,return_tensors='pt')[0]

tensor([    0,  1121,  1844, 37700,  2239,    36, 10644,   574,   238, 14602,
         5448,     2])

In [106]:
model(tokenizer.encode(txt,return_tensors='pt'))[-1][0].shape

torch.Size([1, 12, 56, 513])

In [112]:
att

[tensor([[[[7.1788e-01, 5.5378e-03, 7.5012e-03,  ..., 4.8977e-03,
            3.4527e-03, 6.5412e-03],
           [9.7170e-01, 1.4779e-02, 2.7786e-03,  ..., 6.6864e-05,
            4.3116e-05, 1.1770e-04],
           [6.8943e-01, 1.8883e-01, 2.7626e-02,  ..., 6.6817e-04,
            3.7036e-04, 7.2882e-04],
           ...,
           [7.0390e-02, 6.1729e-03, 9.7764e-03,  ..., 8.8440e-02,
            5.4219e-02, 2.7153e-02],
           [4.7960e-03, 1.3763e-03, 2.0014e-03,  ..., 1.9441e-01,
            1.1273e-01, 8.9215e-02],
           [8.7830e-03, 1.4608e-03, 1.5331e-03,  ..., 1.5334e-01,
            3.3953e-01, 1.7092e-01]],
 
          [[4.0125e-01, 3.1798e-03, 5.1341e-03,  ..., 5.5253e-03,
            1.0862e-01, 9.2635e-03],
           [1.8265e-02, 8.9370e-02, 5.8057e-02,  ..., 5.4733e-03,
            1.0245e-02, 1.0259e-02],
           [3.5664e-03, 1.5641e-02, 4.3318e-02,  ..., 1.6889e-02,
            1.0609e-03, 4.8854e-03],
           ...,
           [3.7154e-03, 2.7398e-03, 3.